In [16]:
import pandas as pd
from Preprocess import * 

In [17]:
dataset_1 = pd.read_csv('data/FakeNewsNet.csv')
dataset_2 = pd.read_csv('data/news_articles.csv')


In [23]:
text = "Je suis testing the preprocess 3ayz ant7r ? yes. BEEB bEep!"
preprocessed_text = preprocess_text(text)
print(preprocessed_text)

example sentence includes stop word number like special character banger batatesmhmra
